# XGBOOST 2020, 2021, 2022

In [24]:
# Deze code is al gerund, dus opnieuw runnen heeft geen zin.

def append_str_to_columns(df, str, exclude_columns):
    new_columns = []
    for column in df.columns:
        if column not in exclude_columns:
            new_columns.append(column + '{}'.format(str))
        else:
            new_columns.append(column)
    df.columns = new_columns
    return df

# stringlist = ["_2019","_2020","_2021","_2022"]
# dflist = ["./data/kwb-2019.xlsx","./data/kwb-2020.xls","./data/kwb-2021.xls", "./data/kwb-2022.xls"]

# exclude_columns = ['gwb_code_10', 'gwb_code_8', 'regio', 'gm_naam', 'recs', 'gwb_code']

# for i in range(len(stringlist)):
#     df = append_str_to_columns(pd.read_excel(dflist[i]), stringlist[i], exclude_columns)
#     savestring = '{}.csv'.format(dflist[i][:15])
#     df.to_csv(savestring, index=False)


# datasets = ["./data/kwb-2021.xls", "./data/kwb-2022.xls"]
# stringlist = ["_2021","_2022"]
# exclude_columns = ['gwb_code_10', 'gwb_code_8', 'regio', 'gm_naam', 'recs', 'gwb_code']

# Dit is ook uitgevoerd dus hoeft niet nog een keer.

zorg21 = zorg21[zorg21['VormenVanJeugdzorg'] == 'T001203']
zorg21 = zorg21[zorg21['Wijken'].str[:2] != 'BU']
kern21 = pd.read_excel('./data/kwb-2021.xls')
kern21 = kern21.applymap(lambda x: x.strip() if isinstance(x, str) else x)
kern21 = kern21[kern21['gwb_code_10'].str[:2] != 'BU']
df = kern21.merge(zorg21, left_on='gwb_code_10', right_on='Wijken', how='left')
df = df.replace('.', np.nan)


df = pd.read_excel("./data/kwb-2022.xls")
zorg = pd.read_csv('./data/zorg22.csv')
zorg = zorg[['Wijken', 'p_jz_tn']]

df = df.drop('p_jz_tn', axis=1)
df = pd.merge(df, zorg, left_on='gwb_code_10', right_on='Wijken', how='left')
df.to_csv("./data/kwb-2022.csv", index=False)

In [20]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

kwb_20 = pd.read_excel('./data/kwb-2020.xls')
zorg21 = pd.read_csv('./data/zorg21.csv')
zorg22 = pd.read_csv('./data/zorg22.csv')
kwb_20 = kwb_20[['gwb_code_10', 'p_jz_tn']]
zorg21 = zorg21[['Wijken', 'p_jz_tn']]
zorg22 = zorg22[['Wijken', 'p_jz_tn']]
df = pd.merge(kwb_20, zorg21, left_on='gwb_code_10', right_on='Wijken', how='inner')
df = pd.merge(df, zorg22, on='Wijken', how='inner')
df.columns = ['gwb_code_10', 'p_jz_tn_20', 'Wijken', 'p_jz_tn_21', 'p_jz_tn_22']

In [ ]:
model = xgb.XGBRegressor()
# print(df['p_jz_tn_20'].value_counts())
# df = df.drop(df[df['p_jz_tn_20'] == '.'].index) # drop rows with '.' in p_jz_tn_20
df['p_jz_tn_20'] = df['p_jz_tn_20'].replace('.', 0) # replace '.' with 0

df['p_jz_tn_20'] = df['p_jz_tn_20'].astype(str)
df['p_jz_tn_20'] = df['p_jz_tn_20'].str.replace(',', '.').astype(float)
X = df[['p_jz_tn_20', 'p_jz_tn_21']]
y = df['p_jz_tn_22']
# y = y[np.isfinite(y)] # drop non-finite values from y
y = np.nan_to_num(y) # fill non-finite values with 0


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse:.4f}')
print(f'R2: {r2:.4f}')

predictions = pd.DataFrame({'Predictions': y_pred})
zorg22_with_predictions = pd.concat([zorg22, predictions], axis=1)
zorg22_with_predictions.to_csv('./data/zorg22_w_pred.csv', index=False)

MSE: 9752.4761
R2: 0.0522
